In [1]:
import nibabel as nib
import numpy as np
import torch
random_b0 = torch.rand((16, 1, 128, 178, 115))
random_residual = torch.rand((16, 1, 128, 178, 115))

print(random_b0.shape)
print(random_residual.shape)

torch.Size([16, 1, 128, 178, 115])
torch.Size([16, 1, 128, 178, 115])


In [ ]:
import sys
sys.path.append("..")
from training import *

b0_cube = grab_cube_around_voxel(image=random_b0, voxel_coordinates=[5, 5, 5], kernel_size=16)
b0_cube = torch.from_numpy(b0_cube).float()

res_cube = grab_cube_around_voxel(image=random_residual, voxel_coordinates=[5, 5, 5], kernel_size=8)
res_cube = torch.from_numpy(res_cube).float()

injection_center = torch.rand((16, 1, 1, 1, 3))
image_coordinates = torch.rand((16, 1, 1, 1, 3))

print(b0_cube.shape)
print(res_cube.shape)
print(injection_center.shape)
print(image_coordinates.shape)

ImportError: attempted relative import with no known parent package

In [96]:
class ResnetEncoder(nn.Module):
    
    # Constructor 
    def __init__(self, input_nc, output_nc=1, ngf=64, n_blocks=6, norm_layer=nn.BatchNorm3d, use_dropout=False, 
                 padding_type='reflect', voxel_wise=False):
        """
        Parameters:
            input_nc (int) -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            ngf (int) -- the number of filters in the last conv layer
            n_blocks (int) -- the number of residual blocks
            norm_layer -- normalization layer
            use_dropout (bool) -- if use dropout layers
            padding_type (str) -- the name of padding layer in conv layers: reflect | replicate | zero
        """

        # Initialize parent class
        super(ResnetEncoder, self).__init__()

        # Initialize the self attributes
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf
        self.n_blocks = n_blocks
        self.norm_layer = norm_layer
        self.use_dropout = use_dropout
        self.padding_type = padding_type
        self.voxel_wise = voxel_wise

        # Whatever this is
        if type(norm_layer) == partial:
            self.use_bias = norm_layer.func == nn.InstanceNorm3d
        else:
            self.use_bias = norm_layer == nn.InstanceNorm3d

        # Define the models
        self.img_model = self.define_img_model()
        self.non_img_model = self.define_non_img_model()
        self.joint_model = self.define_joint_model()

    # Define the model
    def define_img_model(self):
        """
        Define the model architecture
        """

        # Initialize the model and padding size
        model = []
        padding_size = 3
        
        # Define the stride, based on voxel_wise
        if self.voxel_wise:
            stride = 2
        else:
            stride = 1

        # Define the padding layer
        if self.padding_type == 'reflect':
            padding_layer = nn.ReflectionPad3d(padding_size)
        elif self.padding_type == 'replicate':
            padding_layer = nn.ReplicationPad3d(padding_size)
        elif self.padding_type == 'zero':
            padding_layer = nn.ZeroPad3d(padding_size)
        else:
            raise NotImplementedError('padding [%s] is not implemented' % self.padding_type)
        
        # 1. Add the first block
        model.extend([padding_layer, 
                      nn.Conv3d(self.input_nc, self.ngf, kernel_size=7, padding=0, bias=self.use_bias), 
                      self.norm_layer(self.ngf), nn.ReLU(True)])
        
        # 2. Add one convolution
        number_downsampling = 2
        self.number_downsampling = number_downsampling
        mult = 2**number_downsampling
        model += [nn.Conv3d(self.ngf, self.ngf * mult, kernel_size=3,
                    stride=1, padding=1, bias=False),
                        self.norm_layer(self.ngf * mult), nn.ReLU(True)]

        # 3. Add the residual blocks
        for i in range(self.n_blocks):
            model += [ResnetBlock(self.ngf * mult, padding_type=self.padding_type, 
                                  norm_layer=self.norm_layer, use_dropout=self.use_dropout, 
                                  use_bias=self.use_bias)]
            
        # 4. Add more downsampling blocks
        # Cube output: stride 1 | Voxel output: stride 2
        for i in range(number_downsampling):
            mult = 2**(number_downsampling - i)
            model += [nn.Conv3d(self.ngf * mult, int(self.ngf * mult / 2), 
                                kernel_size=3, stride=stride, padding=1, bias=self.use_bias), 
                          self.norm_layer(int(self.ngf * mult / 2)), 
                          nn.ReLU(True)]
            
        # 5. Add another convolutional block for vibes
        # Cube output: stride 1 | Voxel output: stride 2
        model += [nn.Conv3d(int(self.ngf * mult / 2), int(self.ngf * mult / 4),
                            kernel_size=3, stride=stride, padding=1, bias=self.use_bias),
                             self.norm_layer(int(self.ngf * mult / 4)), nn.ReLU(True)]
            
        # 4. Add the last block to make the number of channels as the output_nc and reduce spatial space
        model += [nn.Conv3d(int(self.ngf * mult / 4), self.output_nc, kernel_size=3, stride=2, padding=1, bias=self.use_bias)]
        
        # Cube output: No Adaptive layer | Voxel output: Adaptive layer
        if self.voxel_wise:
            model += [nn.AdaptiveAvgPool3d((1, 1, 1))]
        
        # Return the model
        return nn.Sequential(*model)
    
    # Define the processing for the non-image inputs
    def define_non_img_model(self):
        
        # Stores the model
        model = []
        
        # Add convolutions for the injection centers and image coordinates - expected to have self.output_nc channels
        for i in range(self.number_downsampling):
            model += [nn.Conv3d(self.output_nc, self.output_nc, kernel_size=3, stride=1, padding=1, bias=self.use_bias),
                      self.norm_layer(self.output_nc), 
                          nn.ReLU(True)]
            
        # Return the model
        return nn.Sequential(*model)
            
    # Define joint processing for everything
    def define_joint_model(self):
        
        # Stores the model
        model = []
        
        # Define the factor we multiply by, based on voxel_wise
        if self.voxel_wise:
            factor = 1
        else:
            factor = 3
        
        # Add final convolutions for image and non-image data
        # Cube output: self.output_nc * 3 channels | Voxel output: self.output_nc channels
        for i in range(self.number_downsampling):
            model += [nn.Conv3d(self.output_nc * factor, self.output_nc * factor, kernel_size=3, stride=1, padding=1, 
                                bias=self.use_bias),
                      self.norm_layer(self.output_nc * factor), 
                          nn.ReLU(True)]
            
        # Final convolution to make the number of channels 1
        # Cube output: self.output_nc * 3 channels | Voxel output: self.output_nc channels
        model += [nn.Conv3d(self.output_nc * factor, self.output_nc, kernel_size=3, stride=1, padding=1, bias=self.use_bias)]
        
        # Cube output: No Adaptive layer | Voxel output: Adaptive layer
        if self.voxel_wise:
            model += [nn.AdaptiveAvgPool3d((1, 1, 1))]
            
        # Return the model
        return nn.Sequential(*model)
    
    # Get the normalization layer
    def get_norm_layer(self, norm_layer):

        # If the norm layer is batch norm, we return it
        if "batch" in norm_layer.lower():
            return nn.BatchNorm3d
        elif "instance" in norm_layer.lower():
            return nn.InstanceNorm3d
        else:
            raise NotImplementedError('normalization layer [%s] is not found' % norm_layer)
    
    # Forward pass
    def forward(self, input_x, injection_center, image_coordinates):
        """
        Forward pass
        """
        
        # Define the dimension we concatenate along, depending on voxel wise
        if self.voxel_wise:
            dim = 4
        else:
            dim = 1

        # Do all the convolutions on the cube first
        for layer in self.img_model:
            input_x = layer(input_x)
            print(input_x.shape)
            
        # Do the convolutional layers for the injection center
        injection_center = self.non_img_model(injection_center)
        
        # Do the convolutional layers for the image coordinates
        image_coordinates = self.non_img_model(image_coordinates)
        
        # Concatenate the data along the number of channels
        # Cube output: Dimension 1 | Voxel output: Dimension 4
        input_x = torch.cat((input_x, injection_center), dim=dim)
        input_x = torch.cat((input_x, image_coordinates), dim=dim)
        
        # Do the joint processing
        joint_data = self.joint_model(input_x)
                        
        # Return the model
        return joint_data

In [97]:
import torch
import torch.nn as nn
from models import *

###############################################################
##################### Myronenko Conv Block #####################
###############################################################
class MyronenkoConvolutionBlock(nn.Module):
    
    # Constructor
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0, norm_layer=None, norm_groups=8):
        super(MyronenkoConvolutionBlock, self).__init__()
        self.myronenko_block = self.build_myronenko(in_channels, out_channels, kernel_size, stride, padding, norm_layer, norm_groups)

    # Build the Myronenko block
    def build_myronenko(self, in_channels, out_channels, kernel_size, stride, padding, norm_layer, norm_groups):

        # If norm layer is not specified, then we use Group norm
        if norm_layer is None:
            self.norm_layer = nn.GroupNorm
        else:
            self.norm_layer = norm_layer

        # Set the number of norm groups
        self.norm_groups = norm_groups
        
        # This will hold the Myronenko block
        myronenko_block = []

        # 1. Add the norm layer
        myronenko_block += [self.create_norm_layer(in_channels)]

        # 2. Add the ReLU layer
        myronenko_block += [nn.ReLU(True)]

        # 3. Add the convolutional layer
        myronenko_block += [nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)]

        return nn.Sequential(*myronenko_block)
    
    # Create the normalization layer
    def create_norm_layer(self, in_channels, error_on_non_divisible_norm_groups=False):

        # If the number of in channels is less than the number of norm groups, then we use instance norm
        if in_channels < self.norm_groups:
            return self.norm_layer(in_channels, in_channels)
        # If the number of in channels is divisible by the number of norm groups, then we use group norm
        elif not error_on_non_divisible_norm_groups and (in_channels % self.norm_groups) > 0:
            print("Setting number of norm groups to {} for this convolution block.".format(in_channels))
            return self.norm_layer(in_channels, in_channels)
        # Otherwise, we use group norm
        else:
            return self.norm_layer(self.norm_groups, in_channels)
        
    # Forward pass
    def forward(self, x_input):
        for layer in self.myronenko_block:
            x_input = layer(x_input)
            print("Shape of input in MyronenkoConvBlock: ", x_input.shape)

        return x_input
    
###############################################################
####################### Myronenko Block #######################
###############################################################
class MyronenkoResidualBlock(nn.Module):

    # Constructor
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, norm_layer=None, norm_groups=8):
        super(MyronenkoResidualBlock, self).__init__()
        self.myronenko_block = self.build_myronenko_block(in_channels, out_channels, kernel_size, stride, padding, norm_layer, norm_groups)

    # Build the Myronenko block
    def build_myronenko_block(self, in_channels, out_channels, kernel_size, stride, padding, norm_layer, norm_groups):

        # Define the first convolutional block
        conv_1 = MyronenkoConvolutionBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, 
                                           padding=padding, norm_layer=norm_layer, norm_groups=norm_groups)

        # Define the second convolutional block
        conv_2 = MyronenkoConvolutionBlock(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,
                                             padding=padding, norm_layer=norm_layer, norm_groups=norm_groups)
        
        # If the number of in channels is not equal to the number of our channels, we do 1x1x1 convolution
        if in_channels != out_channels:
            self.sample = conv1x1x1(in_channels, out_channels)
        else:
            self.sample = None
        
        return nn.Sequential(conv_1, conv_2)
    
    # Forward pass
    def forward(self, x_input):

        # Define the identity of x_input, like residual block
        identity = x_input

        # Pass the input through the network
        for layer in self.myronenko_block:
            x_input = layer(x_input)
            print("Shape of input in MyronenkoResBlock: ", x_input.shape)

        # If the sample is not None, then we do 1x1x1 convolution
        if self.sample is not None:
            identity = self.sample(identity)
            print("Identity in res block shape: ", identity.shape)

        # Add the identity to the output
        x_input += identity

        # Return the output
        return x_input

###############################################################
####################### Myronenko Layer #######################
###############################################################
class MyronenkoLayer(nn.Module):

    # Constructor
    def __init__(self, num_blocks, block, in_channels, out_channels, *args, dropout=None, kernel_size=3, **kwargs):
        super(MyronenkoLayer, self).__init__()
        self.myronenko_layer = self.build_myronenko_layer(num_blocks, block, in_channels, out_channels, *args, dropout=dropout, kernel_size=kernel_size, **kwargs)

    # Build the Myronenko layer
    def build_myronenko_layer(self, num_blocks, block, in_channels, out_channels, *args, dropout=None, kernel_size=3, **kwargs):

        # The layer will hold the blocks as a module list
        layer = nn.ModuleList()

        # For number of blocks
        for i in range(num_blocks):

            # Append the block
            layer.append(block(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, *args, **kwargs))

            # Set the in channels to the out channels
            in_channels = out_channels

        # If dropout is not None, then we add dropout
        if dropout is not None:
            self.dropout = nn.Dropout3d(dropout, inplace=True)
        else:
            self.dropout = None

        return layer
    
    # Forward pass
    def forward(self, x_input):
            
        # For each block in the layer
        for index, block in enumerate(self.myronenko_layer):

            # Pass the input through the block
            x_input = block(x_input)
            print("Shape of input in MyronenkoLayer: ", x_input.shape)

            # If dropout is not None, then we add dropout
            if index == 0 and self.dropout is not None:
                x_input = self.dropout(x_input)

        # Return the output
        return x_input


###############################################################
####################### Myronenko Encoder #######################
###############################################################
class MyronenkoEncoder(nn.Module):

    # Constructor
    def __init__(self, in_channels, ngf=32, block_layers=None, layer=MyronenkoLayer,
                 block=MyronenkoResidualBlock, feature_dilation=2, downsampling_stride=2, dropout=0.2, 
                 layer_widths=None, kernel_size=3):
        super(MyronenkoEncoder, self).__init__()
        self.myronenko_encoder = self.build_myronenko_encoder(in_channels, ngf, block_layers, layer, block, feature_dilation,
                                                                downsampling_stride, dropout, layer_widths, kernel_size)
        
    # Build the encoder
    def build_myronenko_encoder(self, in_channels, ngf, block_layers, layer, block, feature_dilation, downsampling_stride, 
                                dropout, layer_widths, kernel_size):
        
        # If the layer blocks are not specified, we use the default ones
        if block_layers is None:
            block_layers = [1, 2, 2, 4]

        # Define layers and downsamples as ModuleLists
        self.layers = nn.ModuleList()
        self.downsampling_convolutions = nn.ModuleList()

        # For every block
        for index, num_blocks in enumerate(block_layers):

            # If the layer widths are not specified, we use the default ones
            if layer_widths is None:
                out_channels = ngf * (feature_dilation ** index)
            else:
                out_channels = layer_widths[index]

            # If dropout is not None, we use it
            if index == 0 and dropout is not None:
                dropout_layer = dropout
            else:
                dropout_layer = None

            # Add the layer to layers
            self.layers.append(layer(num_blocks=num_blocks, block=block, in_channels=in_channels, out_channels=out_channels, 
                                     dropout=dropout_layer, kernel_size=kernel_size))

            # If we're not at the last layer, we add a downsampling convolution
            if index != len(block_layers) - 1:
                self.downsampling_convolutions.append(conv3x3x3(in_channels=out_channels, out_channels=out_channels, 
                                                                stride=downsampling_stride, kernel_size=kernel_size))

            # Print out the layer
            print("Encoder Layer {}:".format(index), in_channels, out_channels)

            # Set the in width to the out width
            in_channels = out_channels


        # Zip the layers and downsampling convolutions together
        encoder = zip(self.layers[:-1], self.downsampling_convolutions)

        # Return the encoder
        return encoder

    # Forward pass
    def forward(self, x_input):

        # For each layer in the encoder
        for layer, downsampling_convolution in self.myronenko_encoder:
                
            # Pass the input through the layer
            x_input = layer(x_input)
            print("Shape of input in MyronenkoEncoder: ", x_input.shape)

            # If the downsampling convolution is not None, then we do 1x1x1 convolution
            if downsampling_convolution is not None:
                x_input = downsampling_convolution(x_input)
                print("Shape of input in MyronenkoEncoder conv: ", x_input.shape)
        
        x_input = self.layers[-1](x_input)
        print("Shape of input in MyronenkoEncoder: ", x_input.shape)

        # Return the output
        return x_input
    
###############################################################
######################## U-Net Encoder ########################
###############################################################
class UNetEncoder(MyronenkoEncoder):

    # Define the forward pass
    def forward(self, x_input):

        # Define the outputs
        outputs = []

        # For each layer in the encoder
        for layer, downsampling_convolution in self.myronenko_encoder:
            
            # Pass the input through the layer
            x_input = layer(x_input)
            print("Shape of input in UNetEncoder: ", x_input.shape)

            # Insert the output into the outputs
            outputs.insert(0, x_input)

            # If the downsampling convolution is not None, then we do 1x1x1 convolution
            if downsampling_convolution is not None:
                x_input = downsampling_convolution(x_input)
                print("Shape of input in downsampling UNetEncoder: ", x_input.shape)

        # Add the last layer to the outputs
        x_input = self.layers[-1](x_input)
        outputs.insert(0, x_input)

        # Return the outputs
        return outputs

###############################################################
####################### Mirrored Decoder #######################
###############################################################
class MirroredDecoder(nn.Module):

    # Constructor
    def __init__(self, ngf=32, block_layers=None, layer=MyronenkoLayer, block=MyronenkoResidualBlock,
                upsampling_scale=2, feature_reduction_scale=2, upsampling_mode="trilinear", align_corners=False,
                layer_widths=None, use_transposed_convolutions=False, kernel_size=3):
        super(MirroredDecoder, self).__init__()
        self.mirrored_decoder = self.build_mirrored_decoder(ngf, block_layers, layer, block, upsampling_scale,
                                                            feature_reduction_scale, upsampling_mode, align_corners,
                                                            layer_widths, use_transposed_convolutions, kernel_size)
        
    # Build the decoder
    def build_mirrored_decoder(self, ngf, block_layers, layer, block, upsampling_scale, feature_reduction_scale,
                               upsampling_mode, align_corners, layer_widths, use_transposed_convolutions, kernel_size):
        
        # If the layer blocks are not specified, we use the default ones
        if block_layers is None:
            self.block_layers = [1, 1, 1, 1]
        else:
            self.block_layers = block_layers

        # Define the widths and feature scales
        self.ngf = ngf
        self.feature_reduction_scale = feature_reduction_scale
        self.layer_widths = layer_widths

        # Define whether or not to use transposed convolutions
        self.use_transposed_convolutions = use_transposed_convolutions

        # Define layers and upsamples as ModuleLists
        self.layers = nn.ModuleList()
        self.pre_upsampling_convolutions = nn.ModuleList()

        # If we use transposed convolutions, define it as a ModuleList
        if use_transposed_convolutions:
            self.upsampling_convolutions = nn.ModuleList()
        else:
            self.upsampling_convolutions = []

        # For every block
        for index, num_blocks in enumerate(self.block_layers):

            # Get the depth of the layer
            depth = len(self.block_layers) - index - 1

            # Calculate the in and out width
            in_channels, out_channels = self.calculate_layer_widths(depth)

            # If the depth isnt 0
            if depth != 0:

                # Append the layer to layers
                self.layers.append(layer(num_blocks=num_blocks, block=block, in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size))

                # If we use transposed convolutions
                if self.use_transposed_convolutions:

                    # Append conv1x1x1 to pre upsampling blocks
                    self.pre_upsampling_convolutions.append(nn.Sequential())

                    # Append the upsampling convolution
                    self.upsampling_convolutions.append(nn.ConvTranspose3d(in_channels, out_channels, kernel_size=kernel_size,
                                                                     stride=upsampling_scale, padding=1))
                else:
                    # Append conv1x1x1 to pre upsampling blocks
                    self.pre_upsampling_convolutions.append(conv1x1x1(in_channels=in_channels, out_channels=out_channels, stride=1))

                    # Append the upsampling convolution
                    self.upsampling_convolutions.append(partial(nn.functional.interpolate, scale_factor=upsampling_scale,
                                                            mode=upsampling_mode, align_corners=align_corners))
            # If the depth is 0
            else:
                # Add layer
                self.layers.append(layer(num_blocks=num_blocks, block=block, in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size))

                # Print out the layer
                print("Decoder Layer {}:".format(index), in_channels, out_channels)

        # Zip the layers and upsampling convolutions together
        decoder = zip(self.pre_upsampling_convolutions, self.upsampling_convolutions, self.layers[:-1])

        # Return the decoder
        return decoder
    
    # Calculate the layer widths
    def calculate_layer_widths(self, depth):
        
        # If the layer widths are specified
        if self.layer_widths is not None:

            # Get the in and out width
            in_channels = self.layer_widths[depth + 1]
            out_channels = self.layer_widths[depth]

        # If the layer widths are not specified, we use the default ones
        else:
            if depth > 0:
                out_channels = int(self.ngf * (self.feature_reduction_scale ** (depth - 1)))
                in_channels = out_channels * self.feature_reduction_scale
            else:
                out_channels = self.ngf
                in_channels = self.ngf

        # Return the in and out width
        return in_channels, out_channels
    
    # Forward pass
    def forward(self, x_input):

        # For each pre upsampling convolution, upsampling convolution, and layer
        for pre_upsampling_convolution, upsampling_convolution, layer in self.mirrored_decoder:

            # Pass the input through the layer
            x_input = layer(x_input)
            print("Shape of input in MirroredDecoder: ", x_input.shape)

            # Pass the input through the pre upsampling convolution
            x_input = pre_upsampling_convolution(x_input)
            print("Shape of preupsampling in MirroredDecoder: ", x_input.shape)

            # Pass the input through the upsampling convolution
            x_input = upsampling_convolution(x_input)
            print("Shape of upsampling in MirroredDecoder: ", x_input.shape)

        x_input = self.layers[-1](x_input)
        print("Shape of concatenated in MirroredDecoder: ", x_input.shape)

        # Return the output
        return x_input

###############################################################
######################## U-Net Decoder ########################
###############################################################
class UNetDecoder(MirroredDecoder):

    # Calculate the layer widths
    def calculate_layer_widths(self, depth):

        # Get them from MirroredDecoder
        in_channels, out_channels = super().calculate_layer_widths(depth=depth)

        # Id the deoth is not at the last block
        if depth != len(self.block_layers) - 1:

            # Double the in width
            in_channels *= 2

        # Print out the layer
        print("Decoder Layer {}:".format(depth), in_channels, out_channels)

        # Return the in and out width
        return in_channels, out_channels
    
    # Define the forward pass
    def forward(self, x_input):

        # x is the first input
        x = x_input[0]

        # For each pre upsampling convolution, upsampling convolution, and layer
        for index, (pre_upsampling_convolution, upsampling_convolution, layer) in enumerate(self.mirrored_decoder):

            # Pass the input through the layer
            x = layer(x)
            print("Decoder input shape at idx {} is: {}".format(index, x.shape))

            # Pass the input through the pre upsampling convolution
            x = pre_upsampling_convolution(x)
            print("Shape of preupsampling in UnetDecoder: ", x.shape)

            # Pass the input through the upsampling convolution
            x = upsampling_convolution(x)
            print("Shape of upsampling in UnetDecoder: ", x.shape)

            # Concatenate
            x = torch.cat((x, x_input[index + 1]), dim=1)
            print("Shape of concatenate in UnetDecoder: ", x.shape)

        # Pass the input through the last layer
        x = self.layers[-1](x)

        # Return the output
        return x

###############################################################
################## Convolutional Autoencoder ##################
###############################################################
class ConvolutionalAutoEncoder(nn.Module):

    # Constructor
    def __init__(self, input_shape=None, in_channels=3, ngf=64, encoder_blocks=None, decoder_blocks=None,
                feature_dilation=2, downsampling_stride=2, interpolation_mode="trilinear",
                encoder_class=MyronenkoEncoder, decoder_class=None, output_channels=None, layer_widths=None,
                decoder_mirrors_encoder=False, activation=None, use_transposed_convolutions=False, kernel_size=3,
                voxel_wise=False):
        super(ConvolutionalAutoEncoder, self).__init__()
        self.convolutional_autoencoder = self.build_convolutional_autoencoder(input_shape, in_channels, ngf, encoder_blocks, decoder_blocks,
                                                                                feature_dilation, downsampling_stride, interpolation_mode,
                                                                                encoder_class, decoder_class, output_channels, layer_widths,
                                                                                decoder_mirrors_encoder, activation, use_transposed_convolutions, 
                                                                                kernel_size, voxel_wise)

    # Build the convolutional autoencoder
    def build_convolutional_autoencoder(self, input_shape, in_channels, ngf, encoder_blocks, decoder_blocks,
                                        feature_dilation, downsampling_stride, interpolation_mode, encoder_class,
                                        decoder_class, output_channels, layer_widths, decoder_mirrors_encoder, activation,
                                        use_transposed_convolutions, kernel_size, voxel_wise):
        
        # If the encoder blocks are not specified, we use the default ones
        if encoder_blocks is None:
            encoder_blocks = [1, 2, 2, 4]

        # Define the attributes of the model
        self.in_channels = in_channels
        self.ngf = ngf
        self.out_channels = output_channels
        self.number_downsampling = 2
        self.norm_layer = nn.BatchNorm3d

        # Define whether it's voxel_wise or not
        self.voxel_wise = voxel_wise

        # Define the image and non-image models
        self.non_img_model = self.define_non_img_model()
        self.joint_model = self.define_joint_model()

        # Define the encoder
        self.encoder = encoder_class(in_channels=in_channels, ngf=ngf, block_layers=encoder_blocks,
                                    feature_dilation=feature_dilation, downsampling_stride=downsampling_stride,
                                    layer_widths=layer_widths, kernel_size=kernel_size)
        
        # Get the decoder class and blocks
        decoder_class, decoder_blocks = self.set_decoder_blocks(decoder_class, encoder_blocks, decoder_mirrors_encoder,
                                                                decoder_blocks)

        # Define the decoder
        self.decoder = decoder_class(ngf=ngf, block_layers=decoder_blocks,
                                     upsampling_scale=downsampling_stride, feature_reduction_scale=feature_dilation,
                                     upsampling_mode=interpolation_mode, layer_widths=layer_widths,
                                     use_transposed_convolutions=use_transposed_convolutions,
                                     kernel_size=kernel_size)
        
        # Set the final convolution
        self.set_final_convolution(output_channels=output_channels)

        # Set the activation
        self.set_activation(activation=activation)

        # Return the convolutional autoencoder
        if self.activation is None:
            return nn.Sequential(self.encoder, self.decoder, self.final_convolution)
        else:
            return nn.Sequential(self.encoder, self.decoder, self.final_convolution, self.activation)

    # Set the decoder blocks
    def set_decoder_blocks(self, decoder_class, encoder_blocks, decoder_mirrors_encoder, decoder_blocks):

        # If the decoder is mirror encoder
        if decoder_mirrors_encoder:
            # The decoder block is the encoder block
            decoder_blocks = encoder_blocks

            # If the decoder class is not specified, we use the MirroredDecoder
            if decoder_class is None:
                decoder_class = MirroredDecoder
            
        # If the deocder blocks is None
        elif decoder_blocks is None:

            # Define it as 1 for every encoder block
            decoder_blocks = [1] * len(encoder_blocks)

            # If the decoder class is not specified, we use the MyronenkoDecoder
            if decoder_class is None:
                decoder_class = MyronenkoDecoder

        # Return the decoder class and blocks
        return decoder_class, decoder_blocks
    
     # Define the processing for the non-image inputs
    def define_non_img_model(self):
        
        # Stores the model
        model = []
        
        # Add convolutions for the injection centers and image coordinates - expected to have self.output_nc channels
        for i in range(self.number_downsampling):
            model += [nn.Conv3d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
                      self.norm_layer(self.out_channels), 
                          nn.ReLU(True)]
            
        # Return the model
        return nn.Sequential(*model)
            
    # Define joint processing for everything
    def define_joint_model(self):
        
        # Stores the model
        model = []
        
        # Define the factor we multiply by, based on voxel_wise
        if self.voxel_wise:
            factor = 1
        else:
            factor = 3
        
        # Add final convolutions for image and non-image data
        # Cube output: self.out_channels * 3 channels | Voxel output: self.out_channels channels
        for i in range(self.number_downsampling):
            model += [nn.Conv3d(self.out_channels * factor, self.out_channels * factor, kernel_size=3, stride=1, padding=1, 
                                bias=False),
                      self.norm_layer(self.out_channels * factor), 
                          nn.ReLU(True)]
            
        # Final convolution to make the number of channels 1
        # Cube output: self.out_channels * 3 channels | Voxel output: self.out_channels channels
        model += [nn.Conv3d(self.out_channels * factor, 1, kernel_size=3, stride=1, padding=1, bias=False)]
        
        # Cube output: No Adaptive layer | Voxel output: Adaptive layer
        if self.voxel_wise:
            model += [nn.AdaptiveAvgPool3d((1, 1, 1))]
            
        # Return the model
        return nn.Sequential(*model)
    
    # Set the final convolution
    def set_final_convolution(self, output_channels):

        # Depending on if it's voxel_wise or not, the final convolution will either just
        # 1. Make # channels to 1
        # 2. Make # channels AND spatial size to 1

        # Cube output: No Adaptive layer | Voxel output: Adaptive layer

        # If it's voxel_wise
        if self.voxel_wise:
            self.final_convolution = nn.Sequential(
                conv1x1x1(in_channels=self.ngf, out_channels=output_channels, stride=2),
                nn.AdaptiveAvgPool3d((1, 1, 1))
            )
        else:
            self.final_convolution = conv1x1x1(in_channels=self.ngf, out_channels=output_channels, stride=2)

    # Set the activation
    def set_activation(self, activation=None):
        # If it's sigmoid
        if activation == "sigmoid":
            self.activation = nn.Sigmoid()
        # If it's softmax
        elif activation == "softmax":
            self.activation = nn.Softmax(dim=1)
        # If it's relu
        elif activation == "relu":
            self.activation = nn.ReLU()
        # If it's leaky relu
        elif activation == "leaky_relu":
            self.activation = nn.LeakyReLU()
        # If it's none
        else:
            self.activation = None

    # Forward
    def forward(self, input_x, injection_center, image_coordinates):

        # Define the dimension we concatenate along, depending on voxel wise
        if self.voxel_wise:
            dim = 4
        else:
            dim = 1

        # Do all the convolutions for the b0 first
        input_x = self.convolutional_autoencoder(input_x)
        print("Shape of x after CAE: ", input_x.shape)

        # Do the convolutional layers for the injection center
        injection_center = self.non_img_model(injection_center)
        print("Shape of injection center after non-img model: ", injection_center.shape)
        
        # Do the convolutional layers for the image coordinates
        image_coordinates = self.non_img_model(image_coordinates)
        print("Shape of image coordinates after non-img model: ", image_coordinates.shape)
        
        # Concatenate the data along the number of channels
        # Cube output: Dimension 1 | Voxel output: Dimension 4
        input_x = torch.cat((input_x, injection_center), dim=dim)
        input_x = torch.cat((input_x, image_coordinates), dim=dim)
        
        # Do the joint processing
        joint_data = self.joint_model(input_x)

        # Return the model
        return joint_data
    
###############################################################
######################## U-Net General ########################
###############################################################
class UNet(ConvolutionalAutoEncoder):

    # Constructor
    def __init__(self, *args, encoder_class=UNetEncoder, decoder_class=UNetDecoder, in_channels=1, out_channels=1,
                 voxel_wise=False, **kwargs):
        super().__init__(*args, encoder_class=encoder_class, decoder_class=decoder_class, in_channels=in_channels,
                         output_channels=out_channels, voxel_wise=voxel_wise, **kwargs)
        self.set_final_convolution(output_channels=out_channels)

In [98]:
model = UNet(voxel_wise=True)

Encoder Layer 0: 1 64
Encoder Layer 1: 64 128
Encoder Layer 2: 128 256
Encoder Layer 3: 256 512
Decoder Layer 3: 512 256
Decoder Layer 2: 512 128
Decoder Layer 1: 256 64
Decoder Layer 0: 128 64
Decoder Layer 3: 128 64


In [99]:
output = model(b0_cube, injection_center, image_coordinates)

print("output shape is: ", output.shape)

Shape of input in MyronenkoConvBlock:  torch.Size([16, 1, 32, 32, 32])
Shape of input in MyronenkoConvBlock:  torch.Size([16, 1, 32, 32, 32])
Shape of input in MyronenkoConvBlock:  torch.Size([16, 64, 32, 32, 32])
Shape of input in MyronenkoResBlock:  torch.Size([16, 64, 32, 32, 32])
Shape of input in MyronenkoConvBlock:  torch.Size([16, 64, 32, 32, 32])
Shape of input in MyronenkoConvBlock:  torch.Size([16, 64, 32, 32, 32])
Shape of input in MyronenkoConvBlock:  torch.Size([16, 64, 32, 32, 32])
Shape of input in MyronenkoResBlock:  torch.Size([16, 64, 32, 32, 32])
Identity in res block shape:  torch.Size([16, 64, 32, 32, 32])
Shape of input in MyronenkoLayer:  torch.Size([16, 64, 32, 32, 32])
Shape of input in UNetEncoder:  torch.Size([16, 64, 32, 32, 32])
Shape of input in downsampling UNetEncoder:  torch.Size([16, 64, 16, 16, 16])
Shape of input in MyronenkoConvBlock:  torch.Size([16, 64, 16, 16, 16])
Shape of input in MyronenkoConvBlock:  torch.Size([16, 64, 16, 16, 16])
Shape of i

In [16]:
model = ResnetEncoder(input_nc=45, ngf=64, n_blocks=3, norm_layer=nn.BatchNorm3d, 
                      padding_type='reflect', voxel_wise=False)
output = model(b0_cube, injection_center, image_coordinates)

print("output shape is: ", output.shape)

AttributeError: type object 'BatchNorm3d' has no attribute 'lower'

In [ ]:
output = model(b0_cube, injection_center, image_coordinates)

print("output shape is: ", output.shape)

In [8]:
# Squeeze the output
output = output.squeeze(0).squeeze(0).detach().numpy()

In [9]:
print(data.shape)
print(output.shape)
print(output)

NameError: name 'data' is not defined

In [ ]:
import glob
import os
import nibabel as nib

def glob_files(PATH_NAME, file_format):
    INPUT_FILES = []
    for file in glob.glob(os.path.join(PATH_NAME, os.path.join("**", "*.{}".format(file_format))), recursive=True):
        INPUT_FILES.append(file)
    return INPUT_FILES

nii_gz_files = glob_files("/notebooks/model_data_w_resize", "nii.gz")
b0_images = [file for file in nii_gz_files if "b0" in file and "resized" not in file]
print(len(b0_images))


In [42]:
import SimpleITK as sitk

reader = sitk.ImageFileReader()
reader.SetFileName(b0_images[0])
image = reader.Execute()

normalizeFilter = sitk.NormalizeImageFilter()
rescaleFilter = sitk.RescaleIntensityImageFilter()
rescaleFilter.SetOutputMaximum(255)
rescaleFilter.SetOutputMinimum(0)

image = normalizeFilter.Execute(image)
image = rescaleFilter.Execute(image)

array = sitk.GetArrayFromImage(image)
print(array.shape)

image_normalized = sitk.GetImageFromArray(array)

sitk.WriteImage(image_normalized, os.path.join(os.getcwd(), "test2.nii.gz"));

(230, 356, 256)


In [4]:
import nibabel as nib
import numpy as np

image = nib.load(b0_images[0])
data = image.get_fdata()

print(data.shape)

normalized_vector = data / np.linalg.norm(data)

print(normalized_vector.shape)

final_img = nib.Nifti1Image(normalized_vector, image.affine)

nib.save(final_img, os.path.join(os.getcwd(), "nibabel.nii"))

(256, 356, 230)
(256, 356, 230)


In [5]:
path = os.path.join(os.getcwd(), "nibabel.nii")

image = nib.load(path)
data = image.get_fdata()

print(data.shape)

(256, 356, 230)


In [16]:
import numpy as np

path_to_residuals = "/notebooks/tract_residuals/predicted_residuals/epoch_1/image_0.npy"
image0 = np.load(path_to_residuals)
print(image0.shape)

(4, 12, 8, 8, 8, 8)


In [26]:
def to_shape(a, shape):
    y_, x_, z_ = shape
    y, x, z = a.shape
    y_pad = (y_-y)
    x_pad = (x_-x)
    z_pad = (z_-z)
    return np.pad(a,((y_pad//2, y_pad//2 + y_pad%2), 
                     (x_pad//2, x_pad//2 + x_pad%2),
                     (z_pad//2, z_pad//2 + z_pad%2)),
                  mode = 'constant')

# Create random array
random_array = np.random.rand(128, 178, 115)
print(random_array.shape)

(128, 178, 115)


In [27]:
# Define kernel size
kernel_size = 8 * 2

# Get the number of values for each axes that need to be added to fit multiple of kernel
padding_needed = [axis % kernel_size for axis in random_array.shape]
print(padding_needed)

output_shape = []
for i in range(random_array.ndim):
    output_shape.append(random_array.shape[i] + padding_needed[i])
    
print(output_shape)

# Padding the random array to the new shape
random_reshaped = to_shape(random_array, output_shape)
print(random_reshaped.shape)

[0, 2, 3]
[128, 180, 118]
(128, 180, 118)


In [10]:
import nibabel as nib
import numpy as np

path = "/notebooks/model_data_w_resize/dMRI_b0/A10-R01_0028-TT21/DWI_concatenated_b0_resized.nii.gz"
injection_center = "/notebooks/model_data_w_resize/injection_centers/A10-R01_0028-TT21/inj_center.csv"
stream_path = "/notebooks/model_data_w_resize/tckmapped_streamlines/A10-R01_0028-TT21/subtracted_unflipped_resized.nii.gz"

# Load the image
image = nib.load(path)
streamline = nib.load(stream_path)
data = image.get_fdata()
stream_data = streamline.get_fdata()
print(data.shape)
print(stream_data.shape)


(128, 178, 115)
(128, 178, 115)


In [13]:
b0_hemi = data[64:, :, :]
res_hemi = stream_data[64:,:,:]

img = nib.Nifti1Image(b0_hemi, affine=np.eye(4))
img2 = nib.Nifti1Image(res_hemi, affine=np.eye(4))

img_b0 = nib.Nifti1Image(data, affine=np.eye(4))
img_str = nib.Nifti1Image(stream_data, affine=np.eye(4))

nib.save(img, "testingcut.nii")
nib.save(img2, "testingcut_res.nii")
nib.save(img_b0, "ogb0.nii")
nib.save(img_str, "ogres.nii")
